# Loan Approval Prediction Kaggle Competition
## October 28, 2024
DICHOSO, Aaron Gabrielle C.

This Notebook is part of a series of notebooks that will contain documentation and methods used for training a SGDClassifier used in the <a href="https://www.kaggle.com/competitions/playground-series-s4e10/"><b>2024 Loan Approval Prediction Kaggle Playground Series</b></a>. 


For this notebook, I will focus on the methods that I utilized for exploratory data analysis, data cleaning, and feature engineering for model training and hyperparameter training later on.

To view the data cleaning itself, feel free to visit the following notebook: 

<ul>
    <li>1. Data Exploration, Cleaning, and Transformations</li>
</ul>



An SGDClassifier is 

## 1. Import Cleaned Datasets

We first import the cleaned datasets from the previous notebook first. In this repository, the cleaned datasets are saved in the <b>./output</b> directory.

In [1]:
##Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

As observed, there are two kinds of training datasets used, the train set without oversampling, and the dataset that underwent ADASYN oversampling. I wish to test the performance of the model comparing these two methods during the hyperparameter tuning phase.

In [2]:
##Import Training Dataset
loans_train_df = pd.read_csv('./outputs/cleaned_loans_train.csv')
loans_train_df.head(5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,PERSON_HOME_OWNERSHIP_MORTGAGE,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,1.569797,-1.081318,-1.896898,-0.578305,0.390423,0.117380,0,1.719062,0,0,...,1,0,0,0,0,0,1,0,0,0
1,-0.921741,-0.052550,0.601227,-0.937769,0.896212,-0.973222,0,-1.364513,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0.240977,-1.508084,0.923860,-0.578305,-0.470628,0.553620,0,1.185873,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.407079,0.435878,1.579649,0.500086,0.277050,0.117380,0,0.087481,0,0,...,1,0,0,0,0,0,0,0,1,0
4,-0.921741,0.098465,-0.486519,-0.578305,-1.318902,-0.646041,0,-0.721995,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
loans_train_ada_df = pd.read_csv('./outputs/cleaned_loans_train_ada.csv')
loans_train_ada_df.head(5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,PERSON_HOME_OWNERSHIP_OTHER,...,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below,loan_status
0,1.569797,-1.081318,-1.896898,-0.578305,0.390423,0.117380,0,1.719062,0,0,...,0,0,0,0,0,1,0,0,0,0
1,-0.921741,-0.052550,0.601227,-0.937769,0.896212,-0.973222,0,-1.364513,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0.240977,-1.508084,0.923860,-0.578305,-0.470628,0.553620,0,1.185873,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.407079,0.435878,1.579649,0.500086,0.277050,0.117380,0,0.087481,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.921741,0.098465,-0.486519,-0.578305,-1.318902,-0.646041,0,-0.721995,0,0,...,0,0,0,0,0,0,0,0,1,0


## 2. Hyperparameter Tuning

The decision tree classifier (DTC) has several hyperparameters that should be tuned to maximize its performance.

In this notebook, I will focus on tuning the following <a href="https://scikit-learn.org/dev/modules/generated/sklearn.tree.DecisionTreeClassifier.html">hyperparameters of the DTC</a> for their explainability, namely:

<ol>
    <li><b>criterion:</b> the formula used for determining the quality of the split performed at a node</li>
    <li><b>splitter:</b> the method used to split each node</li>
    <li><b>max_depth:</b> the maximum depth of the decision tree</li>
    <li><b>min_samples_split:</b> the minimum number of samples required for a node to be able to split</li>
    <li><b>max_features:</b> the maximum number of features to consider when looking for the best way to split a node</li>
    <li><b>oversampling_method:</b> The type of oversampling done in the dataset used.</li>
</ol>

The range of follows I chose for these hyperparameters are as follows:
<ol>
    <li><b>criterion:</b> gini impurity or entropy.</li>
    <li><b>splitter:</b> "best" refers to using the feature that splits the node the best according to the criterion, while "random" refers to the best random split.</li>
    <li><b>max_depth:</b> A range from 1 to 100.</li>
    <li><b>min_samples_split:</b> A fraction, referring to the percentage of samples in the dataset for the minimum number of samples required, with a minimum value of 1e-8.</li>
    <li><b>max_features:</b>"sqrt" refers to using sqrt(number of features) to consider for a split, "log2" refers to using log2(number of features) to consider for a split, and "none" refers to using all features to consider for a split.</li>
        <li><b>oversampling_method:</b> Either using the ADASYN oversampled data set or the unbalanced labels dataset.</li>

</ol>

In [4]:
df_hyper_tuning = pd.DataFrame(columns=['loss', 'alpha', 'eta0', 'tol', 'learning_rate', 'oversampling_method', 'roc_auc'])
df_hyper_tuning

,loss,alpha,eta0,tol,learning_rate,oversampling_method,roc_auc


For the specific method of hyperparameter tuning, I chose to perform bayesian optimization, which is a hyperparameter tuning method that involves observing the past iterations of the tuning process to influence the configurations to test later on. I chose this method over GridSearch because of the numerous possible configurations that I would need to search through not being a feasible method for my system. Additionally, I chose it over RandomSearch because bayesian optimization would be able to utilize my system resources more effectively by searching in areas with higher probabilities of giving me high performances as opposed to randomly testing configurations.

To perform bayesian optimization, I utilized the <a href="https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html">gp_minimize()</a> function provided by the scikit-optimize library.

Following the instructions found in the documentations, I first initialized the search space to be used in the optimization process. This involved creating an array that pertains to the hyperparameters to tune, the data type of the hyperparameters, and the range of possible values to test in the hyperparameter tuning process.

Afterwards, I created the objective function that the gp_minimize() function will execute. The objective function will use the search space defined earlier and test different values for the hyperparameters. It will then return the negative value of the Area Under the ROC (AUC), as this value will be minimized by the gp_minimize() function. Invalid configurations during hyperparameter tuning process will be given a positive value, allowing the bayesian optimization process to avoid such configurations.

I also limit the number of calls performed by the function due to my limited resources.

In [5]:
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import numpy as np
import sys
# Define the search space
search_space = [
    Categorical(['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron'], name='loss'),
    Real(0.0001, 100, name='alpha'),
    Real(0.0, 100.0, name='eta0'),
    Real(0.0001, 0.1, name='tol'),
    Categorical(['constant', 'optimal', 'invscaling', 'adaptive'], name='learning_rate'),
    Categorical(['none', 'ada'], name='oversampling_method')
]

# Define your objective function (e.g., maximizing accuracy)
@use_named_args(search_space)
def objective_function(loss, alpha, eta0, tol, learning_rate, oversampling_method):
    print("================")
    print("Configuration:")
    print("Loss:", loss)
    print("Tolerance:", tol)
    print("Alpha:", alpha)
    print("Eta0:", eta0)
    print("Learning Rate:", learning_rate)
    print("Oversampling Method:", oversampling_method)
    print("----------------")
    try:
        if oversampling_method == 'none':
            X = loans_train_df.loc[:, loans_train_df.columns != "loan_status"]
            y = loans_train_df["loan_status"]
        elif oversampling_method == 'ada':
            X = loans_train_ada_df.loc[:, loans_train_ada_df.columns != "loan_status"]
            y = loans_train_ada_df["loan_status"]
            
        model = SGDClassifier(class_weight='balanced', loss=loss, alpha=alpha, eta0=eta0, max_iter=200, tol=tol, learning_rate=learning_rate)
        roc_auc = cross_val_score(model, X, y, cv=3, scoring='roc_auc').mean()

        print("Results:", -roc_auc)
        print("================")
        df_hyper_tuning.loc[len(df_hyper_tuning.index)] = [loss, alpha, eta0, tol, learning_rate, oversampling_method, roc_auc] 
        return -roc_auc
    except:
        print("Invalid Config")
        return 100000
        

# Perform Bayesian Optimization
res = gp_minimize(objective_function, search_space, n_calls=500)

# Print best parameters
print("Best parameters:", res.x)


Configuration:
Loss: hinge
Tolerance: 0.05412956686808547
Alpha: 84.74747382102345
Eta0: 77.96390028256833
Learning Rate: constant
Oversampling Method: ada
----------------
Results: -0.5217856009318392
Configuration:
Loss: perceptron
Tolerance: 0.039649019886471154
Alpha: 64.41220646667635
Eta0: 39.32627781516281
Learning Rate: constant
Oversampling Method: none
----------------
Results: -0.5503153537364779
Configuration:
Loss: log_loss
Tolerance: 0.023449328206767754
Alpha: 45.96927560370874
Eta0: 75.48336816805381
Learning Rate: constant
Oversampling Method: ada
----------------
Results: -0.595869505490533
Configuration:
Loss: squared_hinge
Tolerance: 0.05056476131422954
Alpha: 52.30544704291901
Eta0: 14.63490172247202
Learning Rate: adaptive
Oversampling Method: ada
----------------
Results: -0.7919216452048573
Configuration:
Loss: modified_huber
Tolerance: 0.021058851618259957
Alpha: 1.8102984788305057
Eta0: 65.79344412146754
Learning Rate: optimal
Oversampling Method: ada
--------

C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_stochastic_gradient.py:723: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Results: -0.7880911176336429


C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['hinge', 0.0001, 100.0, 0.1, 'optimal', 'ada'] before, using random point ['squared_hinge', 90.50282028158718, 6.654634705424524, 0.078463123744636, 'optimal', 'none']
  warnings.warn(


Configuration:
Loss: squared_hinge
Tolerance: 0.078463123744636
Alpha: 90.50282028158718
Eta0: 6.654634705424524
Learning Rate: optimal
Oversampling Method: none
----------------
Results: -0.8609697341185649
Configuration:
Loss: log_loss
Tolerance: 0.09980305527632294
Alpha: 99.34845790365219
Eta0: 14.615092840557523
Learning Rate: optimal
Oversampling Method: none
----------------
Results: -0.8614667588091574
Configuration:
Loss: hinge
Tolerance: 0.04536313958257223
Alpha: 6.2001535312486284
Eta0: 100.0
Learning Rate: optimal
Oversampling Method: none
----------------
Results: -0.8616211004867323
Configuration:
Loss: log_loss
Tolerance: 0.07865158248633465
Alpha: 88.49190039725987
Eta0: 10.208303992631096
Learning Rate: optimal
Oversampling Method: none
----------------
Results: -0.8621446368388171
Configuration:
Loss: hinge
Tolerance: 0.0001
Alpha: 0.0001
Eta0: 100.0
Learning Rate: optimal
Oversampling Method: ada
----------------
Results: -0.8835768289171858
Configuration:
Loss: log

...

Configuration:
Loss: log_loss
Tolerance: 0.04666413063325838
Alpha: 89.86077400115252
Eta0: 81.84668612322838
Learning Rate: constant
Oversampling Method: none
----------------
Results: -0.32098762522791285
Configuration:
Loss: modified_huber
Tolerance: 0.022166575015964994
Alpha: 0.0001
Eta0: 78.16382866283001
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.9040927073815777
Configuration:
Loss: log_loss
Tolerance: 0.0006832334921429543
Alpha: 0.0001
Eta0: 91.98228196577834
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.9046282022259802
Configuration:
Loss: log_loss
Tolerance: 0.0001
Alpha: 0.0001
Eta0: 85.88082121700933
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.9046274019317341
Configuration:
Loss: log_loss
Tolerance: 0.022259731878947524
Alpha: 0.0001
Eta0: 60.22139142264595
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.9046307182677805
Configuration:
L

C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['log_loss', 0.0001, 100.0, 0.0001, 'adaptive', 'none'] before, using random point ['hinge', 77.33255188269172, 87.55481702004381, 0.050349274188254516, 'constant', 'none']
  warnings.warn(


Configuration:
Loss: hinge
Tolerance: 0.050349274188254516
Alpha: 77.33255188269172
Eta0: 87.55481702004381
Learning Rate: constant
Oversampling Method: none
----------------
Results: -0.58200986440154


C:\Users\Aaron\AppData\Roaming\Python\Python311\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['log_loss', 0.0001, 100.0, 0.0001, 'optimal', 'none'] before, using random point ['squared_hinge', 47.66060914633564, 63.42334634734921, 0.008016924546043322, 'constant', 'ada']
  warnings.warn(


Configuration:
Loss: squared_hinge
Tolerance: 0.008016924546043322
Alpha: 47.66060914633564
Eta0: 63.42334634734921
Learning Rate: constant
Oversampling Method: ada
----------------
Results: -0.5
Configuration:
Loss: hinge
Tolerance: 0.061085069793886955
Alpha: 2.804993587496848
Eta0: 81.6332310025716
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.862327704354712
Configuration:
Loss: log_loss
Tolerance: 0.055365206013973094
Alpha: 1.7833655545930158
Eta0: 64.88863968787861
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.8664025735051513
Configuration:
Loss: squared_hinge
Tolerance: 0.07933086612972681
Alpha: 57.707035826049996
Eta0: 50.414394215772894
Learning Rate: adaptive
Oversampling Method: none
----------------
Results: -0.8629726024611158
Configuration:
Loss: modified_huber
Tolerance: 0.008481869207872925
Alpha: 0.0001
Eta0: 91.79000526529029
Learning Rate: adaptive
Oversampling Method: none
----------------
Results:

These tuning results are saved in a dataframe. The hyperparameter tuning results for all models can be viewed in the <b>./hyper_tuning</b> directory.

In [6]:
df_hyper_tuning.sort_values(by=['roc_auc'], ascending=False)

,loss,alpha,eta0,tol,learning_rate,oversampling_method,roc_auc
117,log_loss,0.000100,36.078762,0.004037,adaptive,none,0.904657
72,log_loss,0.000100,100.000000,0.005083,adaptive,none,0.904645
76,log_loss,0.000100,48.278881,0.007399,adaptive,none,0.904645
80,log_loss,0.000100,50.937640,0.007440,adaptive,none,0.904645
159,log_loss,0.000100,87.314916,0.000100,adaptive,none,0.904644
...,...,...,...,...,...,...,...
67,modified_huber,43.190278,51.720587,0.028410,constant,none,0.417533
315,log_loss,74.409319,96.326592,0.013963,constant,ada,0.416761
245,log_loss,76.353531,98.128656,0.012756,constant,ada,0.411720
448,log_loss,71.996511,94.993027,0.016532,constant,ada,0.405449


In [7]:
df_hyper_tuning.to_csv('hyper_tuning/sgd_hyper_tuning.csv', index=False, header=True, encoding='utf-8')

# 3. Results

The results showed that the following configuration produced the best results:

<ol>
    <li><b>criterion: </b>entropy</li>
    <li><b>splitter: </b>best</li>
    <li><b>max_depth: </b>42</li>
    <li><b>min_samples_split: </b>0.042438 = 4.2438%</li>
    <li><b>max_features: </b>log2</li>
    <li><b>oversampling_method: </b>none</li>
</ol>

# 4. Exporting Model

The model with the best configuration found during the hyperparameter tuning process is saved in the <b>./outputs</b> directory.

In [8]:
clf = SGDClassifier(class_weight='balanced', loss=res.x[0], alpha=res.x[1], eta0=res.x[2], max_iter=200, tol=res.x[3], learning_rate=res.x[4])

In [9]:
if res.x[5] == 'none':
    X = loans_train_df.loc[:, loans_train_df.columns != "loan_status"]
    y = loans_train_df["loan_status"]
elif res.x[5] == 'ada':
    X = loans_train_ada_df.loc[:, loans_train_ada_df.columns != "loan_status"]
    y = loans_train_ada_df["loan_status"]
    
clf.fit(X,y)

# Calculate the ROC AUC score
roc_auc = cross_val_score(clf, X, y, cv=3, scoring='roc_auc').mean()
print("Validation AUC:", roc_auc)

Validation AUC: 0.9046316593278497


In [10]:
from joblib import dump
clf.fit(X,y)
dump(clf, './outputs/sgd_model.joblib')

['./outputs/sgd_model.joblib']

# 5. Fitting into Test Data

Finally, we can now generate the predictions made by the DTC on the test data. This is done by isolating the features of the test samples, forwarding it to the DTC for prediction, and appending the predicted class labels with the corresponding IDs of the test data.

In [11]:
##Import Testing Dataset
loans_test_df = pd.read_csv('./outputs/cleaned_loans_test.csv')
loans_test_df

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,58645,-0.755638,0.404383,-0.117198,2.836600,1.455666,2.189522,0,-1.364513,0,...,0,0,0,0,1,0,0,0,0,1
1,58646,-0.257331,1.127233,0.601227,0.140622,0.722635,-0.646041,1,-0.266122,1,...,0,1,0,0,0,0,0,0,0,1
2,58647,-0.257331,-1.418731,0.403331,-0.937769,1.748450,-0.318861,1,-1.364513,0,...,0,0,0,1,0,0,0,0,0,1
3,58648,0.905387,-0.300610,0.169270,-0.398573,-0.470628,-0.209801,0,0.620670,0,...,0,0,0,0,0,0,0,0,1,0
4,58649,-0.257331,1.259932,0.923860,1.039281,1.573370,-0.100741,1,-0.266122,1,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,-0.921741,-1.332883,-0.486519,-1.117500,0.044689,-0.646041,0,-0.266122,1,...,1,0,0,0,0,0,0,0,0,1
39094,97739,-0.921741,-0.389963,0.601227,-0.398573,-1.782989,-0.100741,0,-0.721995,1,...,0,0,0,0,0,0,0,0,0,1
39095,97740,3.895232,0.098465,-1.896898,1.039281,-1.043084,0.989861,0,2.637868,1,...,0,0,0,0,0,0,0,1,0,0
39096,97741,-0.921741,-1.019656,0.169270,0.859550,1.425586,2.516703,1,-0.266122,1,...,0,0,1,0,0,0,0,0,0,1


In [12]:
X_test = loans_test_df.loc[:, loans_test_df.columns != "id"]
X_test

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,PERSON_HOME_OWNERSHIP_MORTGAGE,PERSON_HOME_OWNERSHIP_OTHER,...,LOAN_GRADE_B,LOAN_GRADE_C,LOAN_GRADE_D,LOAN_GRADE_E,LOAN_GRADE_F,LOAN_GRADE_G,CB_PERSON_CRED_HIST_LENGTH_11_17,CB_PERSON_CRED_HIST_LENGTH_18_above,CB_PERSON_CRED_HIST_LENGTH_5_10,CB_PERSON_CRED_HIST_LENGTH_5_below
0,-0.755638,0.404383,-0.117198,2.836600,1.455666,2.189522,0,-1.364513,0,0,...,0,0,0,0,1,0,0,0,0,1
1,-0.257331,1.127233,0.601227,0.140622,0.722635,-0.646041,1,-0.266122,1,0,...,0,1,0,0,0,0,0,0,0,1
2,-0.257331,-1.418731,0.403331,-0.937769,1.748450,-0.318861,1,-1.364513,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0.905387,-0.300610,0.169270,-0.398573,-0.470628,-0.209801,0,0.620670,0,0,...,0,0,0,0,0,0,0,0,1,0
4,-0.257331,1.259932,0.923860,1.039281,1.573370,-0.100741,1,-0.266122,1,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,-0.921741,-1.332883,-0.486519,-1.117500,0.044689,-0.646041,0,-0.266122,1,0,...,1,0,0,0,0,0,0,0,0,1
39094,-0.921741,-0.389963,0.601227,-0.398573,-1.782989,-0.100741,0,-0.721995,1,0,...,0,0,0,0,0,0,0,0,0,1
39095,3.895232,0.098465,-1.896898,1.039281,-1.043084,0.989861,0,2.637868,1,0,...,0,0,0,0,0,0,0,1,0,0
39096,-0.921741,-1.019656,0.169270,0.859550,1.425586,2.516703,1,-0.266122,1,0,...,0,0,1,0,0,0,0,0,0,1


In [13]:
y_pred = clf.predict(X_test)

In [14]:
loans_predictions_df = loans_test_df["id"].copy(deep=True)
loans_predictions_df = loans_predictions_df.to_frame()
loans_predictions_df.insert(1, 'loan_status', y_pred, True)

In [15]:
loans_predictions_df

,id,loan_status
0,58645,1
1,58646,0
2,58647,1
3,58648,0
4,58649,1
...,...,...
39093,97738,0
39094,97739,0
39095,97740,0
39096,97741,1


In [16]:
loans_predictions_df.to_csv('predictions/sgd_predictions.csv', index=False, header=True, encoding='utf-8')